In [ ]:
import logging
import random

from backstrip import backplot
from matplotlib import pyplot as plt
from nbmetalog import nbmetalog as nbm
import pandas as pd
import pecking
from teeplot import teeplot as tp

random.seed(1)

logging.basicConfig(
    format="[%(funcName)s:%(lineno)d] %(message)s",
)
logger = logging.getLogger()


In [ ]:
nbm.print_metadata()


## Get Data


In [ ]:
df = pd.concat(
    [
        pd.read_csv("https://osf.io/n4xyp/download"),
        pd.read_csv("https://osf.io/egjxr/download"),
    ],
    ignore_index=True,
)


In [ ]:
df["Nanoseconds per Op"] = df["Nanoseconds"] / df["Num Operations"]


In [ ]:
def divide_by_subset_mean(group):
    group["Trivial Ratio"] = (
        group["Nanoseconds"]
        / group[
            (group["Policy"] == "trivial")
            & (group["Implementation"] == "surface")
        ]["Nanoseconds"].mean()
    )
    return group


result = (
    df.groupby(["Language"], group_keys=True)
    .apply(divide_by_subset_mean)
    .reset_index(drop=True)
)
result.groupby(["Language", "Implementation", "Policy"])["Trivial Ratio"].agg(
    ["mean", "sem"]
)


## Visualize Results


In [ ]:
tp.tee(
    pecking.peckplot,
    data=df[
        df["Policy"].isin(
            ["steady", "tilted", "trivial"],
        )
    ],
    y="Policy",
    x="Nanoseconds per Op",
    hue="Implementation",
    score="Nanoseconds per Op",
    row="Language",
    y_group="outer",
    sharey=False,
    orient="h",
    aspect=12,
    height=0.8,
    margin_titles=True,
    skimmers=[pecking.skim_lowest],
    skim_labels=["faster"],
    skim_hatches=["*"],
    log_scale=(True, False),
    facet_kws=dict(gridspec_kws={"wspace": 0.2, "hspace": 0.2}),
    teeplot_outattrs={
        "all": "false",
    },
    teeplot_postprocess="""
plt.subplots_adjust(right=0.8);
teed.axes.flat[1].set_ylim(2.5, 0);
teed.set_titles(row_template='{row_name}');
plt.setp(teed.legend.get_texts(), fontsize='8');
plt.setp(teed.legend.get_title(), fontsize='10');""",
)
